## The objective of this problem is to explore different neighbourhoods in the city TORONTO

###### METHOD: We will be using the wickipedia page to extract the data through web scraping techniques with the help of Python requests and beautifulSoup packages. Then,we can get the latitude and longitude coordinates using Python Geocoder Package and we can also map it using folium .  After that,we will use the Foursquare API to explore the neighborhoods.

### Scraping data from wikipedia 

In [13]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import folium
from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
pd.set_option('display.max_columns', None) #to display all the rows of table
pd.set_option('display.max_rows', None)

print("Imported!")

Imported!


In [14]:
!pip install geocoder

You should consider upgrading via the 'c:\python39\python.exe -m pip install --upgrade pip' command.



#### Scraping list of postal codes of toronto using BeautifulSoup

In [15]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data = requests.get(url).text
scrape_data = BeautifulSoup(data, 'lxml')

In [16]:
print(scrape_data.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"f9a6d112-d4be-4532-ac64-051b2dd12d5e","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":1032600019,"wgRevisionId":1032600019,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Communica

In [17]:
column_names=['Postalcode','Borough','Neighborhood']
toronto=pd.DataFrame(columns=column_names)

content=scrape_data.find('div',class_='mw-parser-output')
table=content.table.tbody
postcode=0
borough=0
neighborhood=0

for td in table.find_all('td'):
    postcode=td.p.text[0:3]
    spanTag=td.span
    i=0
    if spanTag.text=="Not assigned":
            pass
    else:    
        for sp in spanTag.find_all('a'):
            if i==0:
                borough=sp.text
                i+=1
            elif i==1:
                neighborhood=sp.text
        toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)            
    

In [18]:
toronto

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M7A,Queen's Park,Lawrence Heights
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Woodbine Gardens
9,M5B,Downtown Toronto,Ryerson


##### Cleaning dataframe

In [19]:
toronto=toronto[toronto.Borough!="Not assigned"]
toronto=toronto[toronto.Borough!=0]
toronto.reset_index(drop=True,inplace=True)

for i in range(toronto.shape[0]):
    if toronto.iloc[i][2]=='Not assigned':
        toronto.iloc[i][2]=toronto.iloc[i][1]
        

In [20]:
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
pd.set_option('display.max_rows',None)
df

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,Rouge
1,M1C,Scarborough,Highland Creek
2,M1E,Scarborough,West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,Birchmount Park
7,M1L,Scarborough,Oakridge
8,M1M,Scarborough,Scarborough Village
9,M1N,Scarborough,Cliffside


In [21]:
df.shape

(103, 3)

In [22]:
df.describe()

,Postalcode,Borough,Neighborhood
count,103,103,103
unique,103,25,78
top,M6B,North York,Downsview
freq,1,25,7


In [23]:
import geocoder # import geocoder

def get_latlng(postalcode):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postalcode))
        lat_lng_coords = g.latlng
    return lat_lng_coords
# Call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(postalcode) for postalcode in df['Postalcode'].tolist()]



In [24]:
# Create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
# Merge the coordinates into the original dataframe
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']
print(df.shape)
df

(103, 5)


,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Rouge,43.81139,-79.19662
1,M1C,Scarborough,Highland Creek,43.78574,-79.15875
2,M1E,Scarborough,West Hill,43.76575,-79.17470
3,M1G,Scarborough,Woburn,43.76812,-79.21761
4,M1H,Scarborough,Cedarbrae,43.76944,-79.23892
5,M1J,Scarborough,Scarborough Village,43.74446,-79.23117
6,M1K,Scarborough,Birchmount Park,43.72582,-79.26461
7,M1L,Scarborough,Oakridge,43.71289,-79.28506
8,M1M,Scarborough,Scarborough Village,43.72360,-79.23496
9,M1N,Scarborough,Cliffside,43.69510,-79.26466


### Creating map 


In [25]:
!pip install geopy
!pip install folium

You should consider upgrading via the 'c:\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\python39\python.exe -m pip install --upgrade pip' command.


In [26]:
address= 'Toronto, Ontario Canada'
geolocator= Nominatim(user_agent="toronto-explorer")
location=geolocator.geocode(address)
latitude=location.latitude
longitude= location.longitude
print("The coordinates of the Toronto are {},{} .".format(latitude,longitude))

The coordinates of the Toronto are 43.6534817,-79.3839347 .


In [27]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)
map_toronto

### Selecting only the boroughs that contain the word TORONTO and will create a map again

In [28]:
df_borough=df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_borough.shape
df_borough.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M4R,North Toronto,Willowdale,43.71458,-79.40668
1,M4W,Downtown Toronto,Rosedale,43.68190,-79.37829
2,M4X,Downtown Toronto,Cabbagetown,43.66788,-79.36649
3,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38133
4,M5A,Downtown Toronto,Harbourfront,43.65512,-79.36264


In [29]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, borough, neighborhood in zip(df_borough['Latitude'], df_borough['Longitude'], df_borough['Borough'], df_borough['Neighborhood']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='red',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)
map_toronto

#### Using the Foursquare API to explore the neighbours

In [30]:
CLIENT_ID = 'ROWO3FEENYIKSG5PLO04KNTQPOAWGOCSWZ1XSJADC1MOIMHY' # Put Your Client Id
CLIENT_SECRET = 'BELNUT4PTLW5D0B50RTTQG00F2XAMWMFQIHGYOIC03TJNTEV' # Put You Client Secret 
VERSION = '20210707'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: Hidden')
print('CLIENT_SECRET: Hidden')

Your credentails:
CLIENT_ID: Hidden
CLIENT_SECRET: Hidden


### Exploring all venues by neighbourhoods and boroughs

In [31]:
def getNearbyVenues(boroughs, neighborhoods, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for borough, neighborhood, lat, lng in zip(boroughs, neighborhoods, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            borough,
            neighborhood, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
                  'Borough',
                  'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
borough_neighborhood_venues = getNearbyVenues(boroughs=df_borough['Borough'],
                                             neighborhoods=df_borough['Neighborhood'],
                                             latitudes=df_borough['Latitude'],
                                             longitudes=df_borough['Longitude'])

In [33]:
borough_neighborhood_venues.head()

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North Toronto,Willowdale,43.71458,-79.40668,St. Clements - Yonge Parkette,43.712062,-79.404255,Park
1,North Toronto,Willowdale,43.71458,-79.40668,Frank's Pizza House,43.712137,-79.409869,Italian Restaurant
2,North Toronto,Willowdale,43.71458,-79.40668,Lytton Park,43.714954,-79.411970,Playground
3,North Toronto,Willowdale,43.71458,-79.40668,NTCC Swimming Pool,43.710553,-79.405786,Gym Pool
4,Downtown Toronto,Rosedale,43.68190,-79.37829,Rosedale Park,43.682328,-79.378934,Playground


In [34]:
borough_neighborhood_venues.shape

(1213, 8)

#### Check how many venues been returned by each neighbour and borough

In [35]:
borough_neighborhood_venues.groupby(['Neighborhood','Borough']).count()

,,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,Borough,,,,,,
Bay Street,Downtown Toronto,81,81,81,81,81,81
Cabbagetown,Downtown Toronto,44,44,44,44,44,44
Church and Wellesley,Downtown Toronto,81,81,81,81,81,81
Design Exchange,Downtown Toronto,100,100,100,100,100,100
Grange Park,Downtown Toronto,49,49,49,49,49,49
Harbourfront,Downtown Toronto,22,22,22,22,22,22
Island airport,Downtown Toronto,78,78,78,78,78,78
King,Downtown Toronto,100,100,100,100,100,100
Rosedale,Downtown Toronto,104,104,104,104,104,104


#### To find out how many unique categories are there in returned venues

In [36]:
print(f'There are {borough_neighborhood_venues["Venue Category"].nunique()} uniques categories.')

There are 191 uniques categories.


##### Analyze each neighborhood and venue


In [37]:
# one hot encoding
toronto_onehot = pd.get_dummies(borough_neighborhood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Borough'] = borough_neighborhood_venues['Borough'] 
toronto_onehot['Neighborhoods'] = borough_neighborhood_venues['Neighborhood'] 
toronto_onehot['Venue'] = borough_neighborhood_venues['Venue'] 

# move neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]



toronto_onehot.head()

,Borough,Neighborhoods,Venue,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Costume Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,North Toronto,Willowdale,St. Clements - Yonge Parkette,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,North Toronto,Willowdale,Frank's Pizza House,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,North Toronto,Willowdale,Lytton Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [38]:
toronto_onehot.shape

(1213, 194)

##### Now , we'll group rows by neighborhood and borough and will take the mean of the frequency of  occurence of each category

In [39]:
toronto_grouped=toronto_onehot.groupby('Neighborhoods').mean().reset_index()
toronto_grouped

,Neighborhoods,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Costume Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Bay Street,0.000000,0.000000,0.000000,0.012346,0.012346,0.000000,0.000000,0.012346,0.000000,0.012346,0.000000,0.000000,0.012346,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012346,0.000000,0.012346,0.000000,0.024691,0.00,0.012346,0.000000,0.000000,0.049383,0.000000,0.012346,0.000000,0.000000,0.000000,0.086420,0.000000,0.111111,0.000000,0.000000,0.00,0.00,0.000000,0.012346,0.000000,0.024691,0.000000,0.000000,0.000000,0.000000,0.012346,0.012346,0.012346,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012346,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.012346,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.012346,0.000000,0.000000,0.000000,0.012346,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.024691,0.000000,0.012346,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.024691,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024691,0.012346,0.000000,0.000000,0.012346,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012346,0.024691,0.012346,0.012346,0.000000,0.000000,0.000000,0.012346,0.000000,0.012346,0.012346,0.012346,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.012346,0.000000,0.000000,0.000000,0.012346,0.012346,0.024691,0.01234

In [40]:
toronto_grouped.shape

(17, 192)

### Printing each neighborhood with top 5  most common venues 

In [41]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhoods']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhoods'] == hood].T.reset_index() #T function is to transpose
    temp.columns = ['venue','freq']
    temp = temp.iloc[2:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
    

----Bay Street----
            venue  freq
0     Coffee Shop  0.11
1  Clothing Store  0.09
2            Café  0.05
3           Hotel  0.02
4  Sandwich Place  0.02


----Cabbagetown----
                venue  freq
0         Coffee Shop  0.09
1                Park  0.07
2  Italian Restaurant  0.05
3                 Pub  0.05
4                Café  0.05


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.11
1  Japanese Restaurant  0.07
2           Restaurant  0.05
3     Sushi Restaurant  0.05
4              Gay Bar  0.04


----Design Exchange----
                venue  freq
0         Coffee Shop  0.12
1               Hotel  0.07
2                Café  0.06
3          Restaurant  0.04
4  Seafood Restaurant  0.03


----Grange Park----
                           venue  freq
0                           Café  0.08
1           Caribbean Restaurant  0.04
2  Vegetarian / Vegan Restaurant  0.04
3                  Grocery Store  0.04
4                    Coffee Sho

##### Writing the function to sort the venues in descending order

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[2:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

##### Creating the new dataframe and displaying top 7 venues for each neighborhood

In [43]:
num_top_venues = 7
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.shape
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,Bay Street,Coffee Shop,Clothing Store,Café,Hotel,Sandwich Place,Grocery Store,Cosmetics Shop
1,Cabbagetown,Coffee Shop,Park,Italian Restaurant,Pub,Café,Pizza Place,Bakery
2,Church and Wellesley,Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Gay Bar,Café,Men's Store
3,Design Exchange,Coffee Shop,Hotel,Café,Restaurant,Seafood Restaurant,Japanese Restaurant,Bakery
4,Grange Park,Café,Caribbean Restaurant,Vegetarian / Vegan Restaurant,Grocery Store,Coffee Shop,Gaming Cafe,Bar


### Clustering the Neighborhoods

In [44]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop([ 'Neighborhoods'], 1)

# run k-means clustering
kmeans = KMeans(init='k-means++', n_clusters=kclusters, random_state=1, max_iter=1000).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:16]

array([0, 2, 3, 3, 2, 4, 2, 3, 3, 0, 0, 3, 3, 2, 3, 3])

#### Creating a new dataframe that contains clusters as well as top 7 venues of each neighborhood

In [45]:
# add clustering labels
neighborhoods_venues_sorted['Cluster Labels'] = kmeans.labels_

toronto_merged = df_borough.copy()

# merge toronto_grouped with df_toronto_borough to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postalcode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,Cluster Labels
0,M4R,North Toronto,Willowdale,43.71458,-79.40668,Playground,Gym Pool,Park,Italian Restaurant,Antique Shop,New American Restaurant,Modern European Restaurant,1
1,M4W,Downtown Toronto,Rosedale,43.68190,-79.37829,Coffee Shop,Hotel,Bar,Restaurant,Asian Restaurant,Park,Italian Restaurant,3
2,M4X,Downtown Toronto,Cabbagetown,43.66788,-79.36649,Coffee Shop,Park,Italian Restaurant,Pub,Café,Pizza Place,Bakery,2
3,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38133,Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Gay Bar,Café,Men's Store,3
4,M5A,Downtown Toronto,Harbourfront,43.65512,-79.36264,Coffee Shop,Breakfast Spot,Yoga Studio,Theater,Electronics Store,Event Space,Pub,4


### visualize this using map

In [46]:

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, neighborhood,cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'],toronto_merged['Neighborhood'],toronto_merged['Cluster Labels']):
    label =  folium.Popup(str(neighborhood) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.5,
        parse_html=False).add_to(map_clusters)
map_clusters

#### Examining the clusters

In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,Cluster Labels
5,Downtown Toronto,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Hotel,Japanese Restaurant,Middle Eastern Restaurant,0
6,Downtown Toronto,Coffee Shop,Clothing Store,Cosmetics Shop,Hotel,Restaurant,Cocktail Bar,Italian Restaurant,0
8,Downtown Toronto,Coffee Shop,Clothing Store,Café,Hotel,Sandwich Place,Grocery Store,Cosmetics Shop,0
18,Downtown Toronto,Coffee Shop,Clothing Store,Café,Hotel,Sandwich Place,Grocery Store,Cosmetics Shop,0


In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,Cluster Labels
0,North Toronto,Playground,Gym Pool,Park,Italian Restaurant,Antique Shop,New American Restaurant,Modern European Restaurant,1


In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,Cluster Labels
2,Downtown Toronto,Coffee Shop,Park,Italian Restaurant,Pub,Café,Pizza Place,Bakery,2
13,Downtown Toronto,Café,Bakery,Coffee Shop,Bookstore,Bar,Restaurant,Gym,2
14,Downtown Toronto,Café,Caribbean Restaurant,Vegetarian / Vegan Restaurant,Grocery Store,Coffee Shop,Gaming Cafe,Bar,2
15,Downtown Toronto,Italian Restaurant,Coffee Shop,Gym / Fitness Center,Café,French Restaurant,Grocery Store,Bar,2


In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,Cluster Labels
1,Downtown Toronto,Coffee Shop,Hotel,Bar,Restaurant,Asian Restaurant,Park,Italian Restaurant,3
3,Downtown Toronto,Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Gay Bar,Café,Men's Store,3
7,Downtown Toronto,Bakery,Coffee Shop,Cocktail Bar,Seafood Restaurant,Pharmacy,Cheese Shop,Beer Bar,3
9,Downtown Toronto,Café,Restaurant,Hotel,Coffee Shop,Gym,Japanese Restaurant,Asian Restaurant,3
10,Downtown Toronto,Coffee Shop,Japanese Restaurant,Hotel,Park,Café,Boat or Ferry,Restaurant,3
11,Downtown Toronto,Coffee Shop,Hotel,Café,Restaurant,Seafood Restaurant,Japanese Restaurant,Bakery,3
12,Downtown Toronto,Coffee Shop,Restaurant,Hotel,Café,Italian Restaurant,Gym,Japanese Restaurant,3
16,Downtown Toronto,Coffee Shop,Hotel,Bar,Restaurant,Asian Restaurant,Park,Italian Restaurant,3
17,Downtown Toronto,Coffee Shop,Hotel,Café,Restaurant,Gym,Deli / Bodega,Bakery,3


In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,Cluster Labels
4,Downtown Toronto,Coffee Shop,Breakfast Spot,Yoga Studio,Theater,Electronics Store,Event Space,Pub,4
